In [1]:
import os
base_dir = '/Users/oliviamurray/Documents/PhD/GitHub/tractTracker'
os.chdir(base_dir)
#comment after installed once
#!git clone https://github.com/MIC-DKFZ/nnUNet.git 
#!git clone https://github.com/prateekgupta891/nnUNet.git #my forked version
#!git clone https://github.com/NVIDIA/apex

In [2]:
respository_dir = os.path.join(base_dir,'nnUNet')
os.chdir(respository_dir)

#!pip install -e .
#(optional installation)
#!pip install --upgrade git+https://github.com/nanohanno/hiddenlayer.git@bugfix/get_trace_graph#egg=hiddenlayer

os.chdir(base_dir)

In [3]:

#libraries
import shutil
from collections import OrderedDict
import json
import numpy as np

#visualization of the dataset
import matplotlib.pyplot as plt
import nibabel as nib

In [4]:
def make_if_dont_exist(folder_path,overwrite=False):
    """
    creates a folder if it does not exists
    input: 
    folder_path : relative path of the folder which needs to be created
    over_write :(default: False) if True overwrite the existing folder 
    """
    if os.path.exists(folder_path):
        
        if not overwrite:
            print(f'{folder_path} exists.')
        else:
            print(f"{folder_path} overwritten")
            shutil.rmtree(folder_path)
            os.makedirs(folder_path)

    else:
      os.makedirs(folder_path)
      print(f"{folder_path} created!")

In [5]:

task_name = 'Task101_IC' #change here for different task name
nnunet_dir = "nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data"
task_folder_name = os.path.join(nnunet_dir,task_name)
train_image_dir = os.path.join(task_folder_name,'imagesTr')
train_label_dir = os.path.join(task_folder_name,'labelsTr')
test_dir = os.path.join(task_folder_name,'imagesTs')
main_dir = os.path.join(base_dir,'nnUNet/nnunet')

In [6]:

make_if_dont_exist(task_folder_name,overwrite = False)
make_if_dont_exist(train_image_dir)
make_if_dont_exist(train_label_dir)
make_if_dont_exist(test_dir,overwrite= False)
make_if_dont_exist(os.path.join(main_dir,'nnunet_trained_models'))

nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task101_IC created!
nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task101_IC/imagesTr created!
nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task101_IC/labelsTr created!
nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task101_IC/imagesTs created!
/Users/oliviamurray/Documents/PhD/GitHub/tractTracker/nnUNet/nnunet/nnunet_trained_models created!


In [7]:

os.environ['nnUNet_raw_data_base'] = os.path.join(main_dir,'nnUNet_raw_data_base')
os.environ['nnUNet_preprocessed'] = os.path.join(main_dir,'preprocessed')
os.environ['RESULTS_FOLDER'] = os.path.join(main_dir,'nnUNet_trained_models')

In [8]:
def copy_and_rename(old_location,old_file_name,new_location,new_filename,delete_original = False):

    shutil.copy(os.path.join(old_location,old_file_name),new_location)
    os.rename(os.path.join(new_location,old_file_name),os.path.join(new_location,new_filename))
    if delete_original:
        os.remove(os.path.join(old_location,old_file_name))